- Don't forget to take into account utilities by looking at the site for each apartment building
- only grab lastest scrape to send as update

In [22]:
import glob
import pandas as pd
import numpy as np

In [56]:
filenames = glob.glob("csvs/*.csv")

## Calibre

In [85]:
calibre = pd.read_csv(filenames[1])

In [92]:
def calibreApts(calibre):
    aptsOfInterest = calibre[(~calibre['floorplan_name'].str.contains("Hampton")) & (calibre['rent_min'] < 1750)]
    groupbyColumns = aptsOfInterest.columns[:-3].tolist() + [aptsOfInterest.columns[-1]] + [aptsOfInterest.columns[-3]]
    aptsFormatted = aptsOfInterest.groupby(groupbyColumns).agg(start_move_in_day=('move_in_day', 'min'),
                                                               end_move_in_day=('move_in_day', 'max'))
    return aptsFormatted

In [93]:
calibreApts(calibre)

,,,,,,,,,,start_move_in_day,end_move_in_day
scraping_date,scraping_time,floorplan_name,bedroom,bathroom,squareFootage,rent_min,rent_max,move_in_year,move_in_month,,
2022/05/20,12,Fillmore Classic II,1.0,1.0,1040.0,1640,1680,2022,5,20,30


## Legacy

In [78]:
legacy = pd.read_csv(filenames[0])

In [101]:
def legacyApts(legacy):
    aptsOfInterest = legacy[(legacy['bedroom'] >= 2) & (legacy['rent_min'] < 1750)]
    groupbyColumns = aptsOfInterest.columns[:6].tolist() + [aptsOfInterest.columns[-1]] + [aptsOfInterest.columns[-3]]
    aptsFormatted = aptsOfInterest.groupby(groupbyColumns).agg(days_range_rent_min_min=('rent_min', 'min'),
                                                               days_range_rent_max_max=('rent_max', 'max'),
                                                               start_move_in_day=('move_in_day', 'min'),
                                                               end_move_in_day=('move_in_day', 'max'))
    return aptsFormatted

In [112]:
legacyApts(legacy).sort_values(['days_range_rent_min_min', 'floorplan_name'])

days_range_rent_min_min  \
scraping_date scraping_time floorplan_name bedroom bathroom squareFootage move_in_year move_in_month                            
2022/05/20    12            b1dh           2.0     1.0      980.0         2022         6                                 1381   
                            b1dh_e         2.0     1.0      980.0         2022         5                                 1398   
                            b1dh           2.0     1.0      980.0         2022         5                                 1417   
                            b1dh_e         2.0     1.0      980.0         2022         6                                 1437   
                            bt_e           2.0     1.5      1100.0        2022         5                                 1565   
                                                                                       6                                 1565   
                            b2dh_p         2.0     1.5      1000.0        2022         5                                 1576   
                                                                                       6                                 1576   
                            bt_e           2.0     1.5      1100.0        2022         7                                 1728   
                            b3             2.0     2.0      980.0         2022         5                                 1730   
                                                                                       6                                 1730   

                                                                                                      days_range_rent_max_max  \
scraping_date scraping_time floorplan_name bedroom bathroom squareFootage move_in_year move_in_month                            
2022/05/20    12            b1dh           2.0     1.0      980.0         2022         6                                 2033   
                            b1dh_e         2.0     1.0      980.0         2022         5                                 1975   
                            b1dh           2.0     1.0      980.0         2022         5                                 1953   
                            b1dh_e         2.0     1.0      980.0         2022         6                                 2057   
                            bt_e           2.0     1.5      1100.0        2022         5                                 1966   
                                                                                       6                                 2347   
                            b2dh_p         2.0     1.5      1000.0        2022         5                                 1930   
                                                                                       6                                 2016   
                            bt_e           2.0     1.5      1100.0        2022         7                                 2404   
                            b3             2.0     2.0      980.0         2022         5                                 2201   
                                                                                       6                                 2336   

                                                                                                      start_move_in_day  \
scraping_date scraping_time floorplan_name bedroom bathroom squareFootage move_in_year move_in_month                      
2022/05/20    12            b1dh           2.0     1.0      980.0         2022         6                              1   
                            b1dh_e         2.0     1.0      980.0         2022         5                             20   
                            b1dh           2.0     1.0      980.0         2022         5                             20   
                            b1dh_e         2.0     1.0      980.0         2022         6                              1   
        

## Madison

In [103]:
madison = pd.read_csv(filenames[2])legacyApts

In [105]:
madison[(madison['bedroom'] >= 2) & (madison['rent_min'] < 1750)].drop_duplicates()

,scraping_date,scraping_time,floorplan_name,bedroom,bathroom,apartment_number,square_footage,rent_min,rent_max,move_in_month,move_in_day,move_in_year
9,2022/05/20,12,Cypress,2,2,2027,987,1708,2188,5,20,2022
